In [ ]:
"""
从新浪财经上下载年报：

优先==》纯html，下载速度快
缺点==》只有2008年后的数据，并且有的公司还没有，数据不是太全
"""

In [1]:
#!jupyter nbconvert --to script web_spider3_sina.ipynb
import json
pdf_needs = {}
with open('./pdf_status.json') as f:
    pdf_needs = json.load(f) # {'123456_2001':[pdf,txt,http1,http2,http3]}
code_2_years = {} #{'00001':[]}
YEAR='all'
counter = 0
c2 = 0
for code_year,data in pdf_needs.items():
    code,year = code_year.split('_')
    if int(year)<=2009: # 新浪只有最近10年的数据
        continue
    if not(YEAR=='all' or year in YEAR):
        continue
    c2 += 1
    if not(data[0]<100 and data[1]<20):
        continue
#     print(code_year,data)
    counter +=1
    if code not in code_2_years:
        code_2_years[code]=[]
    code_2_years[code].append(year)
    
print(f'maybe find by sina is {counter},code num={len(code_2_years.keys())},c2={c2}') 

# print(code_2_years)

maybe find by sina is 694,code num=482,c2=20497


In [2]:
import os
import pandas as pd
import bs4
from urllib.request import urlopen
from bs4 import BeautifulSoup as soup
import re
from tqdm import tqdm

url_prefix = 'http://vip.stock.finance.sina.com.cn'
url_pattern = r'href=".+target='
title_pattern = r'_blank">.+</a>'
year_pattern = r'20[01][0-9]'

code = '000001'

import re
import requests
yearReport_regex = re.compile('摘要|取消|英文|补充公告|关于调整|的公告')
def checkIsChsYearReport(fileName):
    flag = True
    if yearReport_regex.search(fileName): # 此处用用match，match不支持中文
        flag = False
    if not(fileName.find('年报')>=0 or fileName.find('年度报告') >=0 ):
        flag = False
#     if flag:
# #         print('ok',' '*20,fileName)
#         pass
#     else:
#         print('not ok',' '*16,fileName)
    return flag


def getTitle(code):
    try:
        reports ={}
        company_url = f'http://vip.stock.finance.sina.com.cn/corp/go.php/vCB_Bulletin/stockid/{code}/page_type/ndbg.phtml'
        print(f'Processing {code} ')
        headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'      
                  }
        r = requests.get(company_url,headers=headers)
        r.encoding = 'gbk'
        page = r.text
#         print(page)

        page_soup = soup(page, "html.parser")
        content = page_soup.find('div',{'class':'datelist'})

        all_url_title = content.findAll('a')

        current_id_set = set()
        for v in all_url_title:
#             print(type(v),v)
            if not checkIsChsYearReport(str(v)):
                continue
#             print('a')
            url = url_prefix + re.findall(url_pattern,str(v))[0][6:-9]
            url = url.replace('amp;','')          

            title = re.findall(title_pattern,str(v))[0][8:-4]
            year = re.findall(year_pattern, title)
            if not year: continue
            year = year[0]
            report_id = code+'_'+year
            if report_id not in reports:
#                 print(report_id,v)
                reports[report_id]=url;
                
#         print(reports)
        if len(reports.keys())<=0:
            print(f'没有找到任何年报信息',page[:2048])
        return reports
    except Exception as e:
        print(f"getTitle({code})",e)

# print(getTitle(code))  

def readContent(url):
    try:
        page_client = urlopen(url)
        page = page_client.read().decode('gbk')
        page_client.close()

        report_soup = soup(page, "html.parser", )

        content = str(report_soup.find('pre'))
#         print(content)
        return content
    except Exception as e:
        print(e)

# url = 'http://vip.stock.finance.sina.com.cn/corp/view/vCB_AllBulletinDetail.php?stockid=000004&id=4386458'; 
# readContent(url)

In [3]:
url = 'http://vip.stock.finance.sina.com.cn/corp/view/vCB_AllBulletinDetail.php?stockid=000004&id=4386458'; 
url='http://vip.stock.finance.sina.com.cn/corp/view/vCB_AllBulletinDetail.php?stockid=002565&id=4333220'

def get_content_pdf_url(url): 
    try:
        headers = {
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_13_5) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/73.0.3683.103 Safari/537.36'      
                  }
        r = requests.get(url,headers=headers)
        r.encoding = 'gbk'
        page = r.text
        report_soup = soup(page, "html.parser", )
        content = str(report_soup.find('pre'))
        pdf_url=None
        for link in report_soup.find_all('a'):
            if link.string=='公告原文':
                pdf_url = link.get('href')
        if pdf_url is None:
            print(f'get pdf_url failed,{url}')
        return content,pdf_url
    except Exception as e:
        print(e)
        return None,None
#get_content_pdf_url(url)

In [5]:
#!jupyter nbconvert --to script down_pdf.ipynb

import requests
#url = 'http://www.sse.com.cn/disclosure/listedinfo/announcement/c/2010-03-31/600795_2009_n.pdf'
# 对于szse.cn 有点特殊，有的用download可以下载，有的不能
url = 'http://disc.static.szse.cn/download/disc/disk01/finalpage/2008-04-19/cda5e777-cc3c-4934-9971-99ce9e4f6fe0.PDF'
url = 'http://disc.static.szse.cn/disc/disk01/finalpage/2008-04-19/cda5e777-cc3c-4934-9971-99ce9e4f6fe0.PDF'
def down_pdf(filename,url):
    success = True
    try:
        r = requests.get(url, stream=True)
        with open(filename, 'wb') as fd:
            for chunk in r.iter_content(chunk_size=102400):
                fd.write(chunk)
    except Exception as e:
        success = False
        print(f'download {url} faild',e)
        pass
    
    return filename,success

import subprocess
import os 
import sys
def execute(command):
#     print(command," ".join(command))
#     " ".join(command)
    process = subprocess.Popen("  ".join(command), shell=True, stdout=subprocess.PIPE, stderr=subprocess.STDOUT)
    # Poll process for new output until finished
    while True:
        if process.poll() is not None:
            break
        nextline = process.stdout.readline()
        if type(nextline) is bytes:
            nextline = nextline.decode("utf-8") 
        sys.stdout.write(nextline)
        sys.stdout.flush()
        if process.poll() is not None:
            break

    output = process.communicate()[0]
    exitCode = process.returncode

    if (exitCode == 0):
        return output
    else:
        raise ProcessException(command, exitCode, output)

def down_pdf2(filename,url):

    success = True
    try:
        if os.path.exists(filename):
            os.remove(filename)
        # --no-use-server-timestamps  不根据服务器上的时间戳设置本地文件的时间戳
        execute(["wget","--no-use-server-timestamps","-O",filename ,url])
    except Exception as e:
        success = False
        print(f'download {url} faild',e)
        pass
    
    return filename,success

# down_pdf2("xx.pdf",url)

In [6]:
import os
import time
counter=0

for code,years in tqdm(code_2_years.items()):
    print(code,years)
    time.sleep(1)
    reports = getTitle(code)
#     print(reports)
    retry = 0
#     while not reports and retry<3:
#         print(f"获取年报地址失败:{code}, retrying..")
#         time.sleep(1+1*retry*retry)
#         reports = getTitle(code)
#         retry += 1

    if not reports:
        print(f"获取年报地址失败:{code}",'*'*50)
        with open('error.txt','a') as f2:
            f2.write(f'{code}\n')
            f2.close()
        continue
    print('a'*10)
    for year in years:
        url = None
        for (f,_url) in reports.items():
            if f[-4:] == year:
                url = _url
        if url is None:
            print(f'{"*"*80}{code}_{year} has not find ...{reports.keys()}')
            continue
        file = f"../txt/{year}/{code}_{year}.txt"
        if os.path.exists(file):
            print(f'{file},size={os.path.getsize(file)}')
#             continue
        print(f"开始下载:{code}_{year}, url={url}")
        content,pdf_url = get_content_pdf_url(url)
        retry = 0
        while not content and retry<3:
            print(f"获取年报地址失败:{i}, retrying..")
            time.sleep(0.1+0.5*retry*retry)
            content,pdf_url = get_content_pdf_url(url)
            retry += 1
        if not content:
            print( f"{'-'*50}下载文件失败:")
            continue
        if len(content)<150*1024:
            print( f"{'x'*50}下载文件成功:但文件太小content={int(len(content)/1024)}KB")
            down_pdf2(f"../pdf/{year}/{code}_{year}.pdf",pdf_url)
            continue
                
        print(f"{'+'*50}下载文件成功:content={int(len(content)/1024)}KB,to save {file}")
        with open(file,'w') as f:
            counter += 1
            f.write(content)
        
        if counter %100 ==0:
            print('*'*80)
            print('下载文件数',counter)
            print('*'*80)
        

  0%|          | 0/482 [00:00<?, ?it/s]

002135 ['2010', '2011']
Processing 002135 


  0%|          | 1/482 [00:01<09:47,  1.22s/it]

aaaaaaaaaa
********************************************************************************002135_2010 has not find ...dict_keys(['002135_2018', '002135_2017', '002135_2016', '002135_2015', '002135_2014', '002135_2013', '002135_2012'])
********************************************************************************002135_2011 has not find ...dict_keys(['002135_2018', '002135_2017', '002135_2016', '002135_2015', '002135_2014', '002135_2013', '002135_2012'])
002138 ['2010']
Processing 002138 


  0%|          | 2/482 [00:04<14:37,  1.83s/it]

aaaaaaaaaa
开始下载:002138_2010, url=http://vip.stock.finance.sina.com.cn/corp/view/vCB_AllBulletinDetail.php?stockid=002138&id=660663
get pdf_url failed,http://vip.stock.finance.sina.com.cn/corp/view/vCB_AllBulletinDetail.php?stockid=002138&id=660663
xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx下载文件成功:但文件太小content=3KB
download None faild sequence item 4: expected str instance, NoneType found
002141 ['2010']
Processing 002141 
aaaaaaaaaa
开始下载:002141_2010, url=http://vip.stock.finance.sina.com.cn/corp/view/vCB_AllBulletinDetail.php?stockid=002141&id=676306


KeyboardInterrupt: 